In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div align='center'><font size="6" color="#43C6DB">SIIM-ISIC Melanoma Classification </font></div>
<hr>

![](https://higherlogicdownload.s3.amazonaws.com/SITCANCER/2c19e5a6-3adb-4d01-b46c-c01e11745b3a/UploadedImages/Education/Patient_CONNECT/Melanoma_Patient_Resource_Guide/Stages_of_Melnoma.jpg)
source:https://higherlogicdownload.s3.amazonaws.com/SITCANCER/2c19e5a6-3adb-4d01-b46c-c01e11745b3a/UploadedImages/Education/Patient_CONNECT/Melanoma_Patient_Resource_Guide/Stages_of_Melnoma.jpg


## Objective

In this competition, the goal is to identify melanoma in images of skin lesions. There are images within the same patient and and the task is to determine which are likely to represent a melanoma. Value 0 denotes benign, and 1 indicates malignant. Using patient-level contextual information may help the development of image analysis tools, which could better support clinical dermatologists.

Melanoma is a deadly disease, but if caught early, most melanomas can be cured with minor surgery. Image analysis tools that automate the diagnosis of melanoma will improve dermatologists' diagnostic accuracy. Better detection of melanoma has the opportunity to positively impact millions of people.


## Evaluation Metric

![](https://i.pinimg.com/736x/dd/aa/f1/ddaaf1a24e601be6085e8b2ef6c7d819.jpg)
source: https://i.pinimg.com/736x/dd/aa/f1/ddaaf1a24e601be6085e8b2ef6c7d819.jpg



References: 

https://www.kaggle.com/parulpandey/melanoma-classification-eda-starter

https://www.kaggle.com/tunguz/melanoma-classification-eda-and-modeling



# Prepare workspace


### Upload Libraries

In [ ]:

import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statistics
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import logit
from scipy.stats import chi2_contingency
from scipy.stats import kurtosis 
from scipy.stats import skew
from statistics import stdev 
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')


### Upload data sets

In [ ]:

train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')


# Summarize Data

In [ ]:
# Look at dimension of data set and types of each attribute
train.info()

In [ ]:
test.info()

In [ ]:
# Summarize attribute distributions of the data frame
train.describe(include='all')

In [ ]:
test.describe(include='all')

In [ ]:
# Take a peek at the first rows of the data
train.head(10)

In [ ]:
test.head(10)

The number of unique patients is less than the total number of patients, so each patient have multiple images.

# Exploratory Data Analysis

### Handling Missing Values

In [ ]:
# Check missing values both to numeric features and categorical features
train.isnull().sum()/train.shape[0]*100

In [ ]:
test.isnull().sum()/test.shape[0]*100

In [ ]:
# Data Imputation
# Input missing values with median or mode depending of features class
train['sex'].fillna(train['sex'].mode()[0], inplace=True)
train['age_approx'].fillna(train['age_approx'].median(), inplace=True)
train['anatom_site_general_challenge'].fillna(train['anatom_site_general_challenge'].mode()[0], inplace=True)
test['anatom_site_general_challenge'].fillna(test['anatom_site_general_challenge'].mode()[0], inplace=True)



# Target Variable Analysis

The target variable is grouped into two classes: 0 = benign and 1 = malignant. Looking at the barplot, it's higly imbalanced.

In [ ]:
# Summarize the class distribution 
count = pd.crosstab(index = train['target'], columns="count")
percentage = pd.crosstab(index = train['target'], columns="frequency")/pd.crosstab(index = train['target'], columns="frequency").sum()
pd.concat([count, percentage], axis=1)

In [ ]:
# Plot the target variable
ax = sns.countplot(x=train['target'], data=train, order=[0,1]).set_title("Target Variable Distribution")

### Numerical Features Analysis

In [ ]:
# Univariate analysis looking at Standard Deviation, Skewness and Kurtosis for train set

print('\nStandard Deviation :', stdev(train['age_approx']), 
      '\nSkewness :', skew(train['age_approx']), 
        '\nKurtosis :', kurtosis(train['age_approx']))

In [ ]:
# Univariate analysis looking at Standard Deviation, Skewness and Kurtosis for test set

print('\nStandard Deviation :', stdev(test['age_approx']), 
      '\nSkewness :', skew(test['age_approx']), 
        '\nKurtosis :', kurtosis(test['age_approx']))

In [ ]:
# graphical function for univariate analysis

def num_plot(dataframe, feature):
    plt.figure(figsize=(15, 5))

    # histogram
    plt.subplot(1, 3, 1)
    sns.distplot(train[feature], bins=30, color='g')
    plt.title('Histogram')
    # Q-Q plot
    plt.subplot(1, 3, 2)
    stats.probplot(train[feature], dist="norm", plot=plt)
    plt.ylabel('Variable quantiles')
    # boxplot
    plt.subplot(1, 3, 3)
    x=train[feature]
    sns.boxplot(x,linewidth=1.5, color='g')
    plt.title('Boxplot')

    plt.show()

In [ ]:
# train
# age_approx
num_plot(train, 'age_approx')

In [ ]:
# test
# age_approx
num_plot(test, 'age_approx')

In [ ]:
# Feature selection with Kendall's Test

alpha = 0.05
var = 'age_approx'
p = stats.kendalltau(train['target'],train[var])[1]
if p <= alpha:
    print('{0} Dependent (reject H0)'.format(var))
else:
    print('{0} Independent (fail to reject H0)'.format(var))

### Categorical Features Analysis

In [ ]:
# Univariate analysis with frequency and barplots for train set
sns.set( rc = {'figure.figsize': (5, 5)})
fcat_tr = ['sex','anatom_site_general_challenge','diagnosis','benign_malignant']

for col in fcat_tr:
    count = pd.crosstab(index = train[col], columns="count")
    percentage = pd.crosstab(index = train[col], columns="frequency")/pd.crosstab(index = train[col], columns="frequency").sum()
    tab = pd.concat([count, percentage], axis=1)
    plt.figure()
    sns.countplot(x=train[col], data=train, palette="Set1")
    plt.xticks(rotation=45)
    print(tab)
    plt.show()

In [ ]:
# Univariate analysis with frequency and barplots for test set
sns.set( rc = {'figure.figsize': (5, 5)})
fcat_te = ['sex','anatom_site_general_challenge']

for col in fcat_te:
    count = pd.crosstab(index = test[col], columns="count")
    percentage = pd.crosstab(index = test[col], columns="frequency")/pd.crosstab(index = test[col], columns="frequency").sum()
    tab = pd.concat([count, percentage], axis=1)
    plt.figure()
    sns.countplot(x=test[col], data=test, palette="Set1")
    plt.xticks(rotation=45)
    print(tab)
    plt.show()

In [ ]:
# Bivariate analysis with barplots for train set
sns.set( rc = {'figure.figsize': (5, 5)})

for col in fcat_tr:
    plt.figure()
    sns.countplot(x=train[col], hue=train['target'], data=train, palette="Set2")
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Feature Selection with Chi-Square Test 
alpha = 0.05
for var in fcat_tr:
    X = train[var].astype(str)
    Y = train['target'].astype(str)
    dfObserved = pd.crosstab(Y,X)
    chi2, p, dof, expected = stats.chi2_contingency(dfObserved.values)
    if p <= alpha:
    	print('{0} Dependent (reject H0)'.format(var))
    else:
        print('{0} Independent (fail to reject H0)'.format(var))

### Visualising a keep of images

In [ ]:
img = []
for i, image_id in enumerate(tqdm(train['image_name'].head(20))):
    im = Image.open(f'../input/siim-isic-melanoma-classification/jpeg/train/{image_id}.jpg')
    im = im.resize((128, )*2)
    img.append(im)

In [ ]:
img[0]

In [ ]:
img[5]

In [ ]:
img[10]

In [ ]:
img[15]

### Visualizing Images with Benign lesions

In [ ]:
benign = train[train['benign_malignant']=='benign']
malign = train[train['benign_malignant']=='malignant']

In [ ]:
img_b = []
for i, image_id in enumerate(tqdm(benign['image_name'].tail(20))):
    im = Image.open(f'../input/siim-isic-melanoma-classification/jpeg/train/{image_id}.jpg')
    im = im.resize((128, )*2)
    img_b.append(im)

In [ ]:
img_b[1]

In [ ]:
img_b[6]

In [ ]:
img_b[11]

In [ ]:
img_b[16]

### Visualizing Images with Malignant lesions

In [ ]:
img_m = []
for i, image_id in enumerate(tqdm(malign['image_name'].head(20))):
    im = Image.open(f'../input/siim-isic-melanoma-classification/jpeg/train/{image_id}.jpg')
    im = im.resize((128, )*2)
    img_m.append(im)

In [ ]:
img_m[2]

In [ ]:
img[7]

In [ ]:
img[12]

In [ ]:
img[17]